In [1]:
import json
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import argrelextrema

In [2]:
def safe_float_array(seq):
    """Convert list-like to float numpy array, skipping non-numeric entries."""
    out = []
    for v in seq:
        try:
            out.append(float(v))
        except (ValueError, TypeError):
            continue
    return np.array(out, dtype=float)

def interp_on_common_grid(subplots, x_common):
    """Interpolate each replica to x_common, skipping invalid ones."""
    reps = []
    for rep_data in subplots.values():
        x = safe_float_array(rep_data["xdata"])
        y = safe_float_array(rep_data["ydata"])
        if len(x) == 0 or len(y) == 0:
            continue
        # sort by x and collapse duplicates if any
        order = np.argsort(x)
        x_sorted = x[order]
        y_sorted = y[order]
        if np.any(np.diff(x_sorted) == 0):
            uniq_x, idx = np.unique(x_sorted, return_inverse=True)
            sums = np.zeros_like(uniq_x, dtype=float)
            counts = np.zeros_like(uniq_x, dtype=int)
            for k, j in enumerate(idx):
                sums[j] += y_sorted[k]
                counts[j] += 1
            xu = uniq_x
            yu = sums / counts
        else:
            xu, yu = x_sorted, y_sorted

        # need at least 2 points to interpolate
        if xu.size >= 2:
            reps.append(np.interp(x_common, xu, yu))
    return reps

In [3]:
# --- Load JSON file ---
with open("qaf.PlotData.json", "r") as f:
    data = json.load(f)

targets = {"dgde", "dgde_unorm", "dgde_unnorm"}  # accepted names

In [4]:
# --- Iterate over all plots ---
for plot_name, plot_data in data.items():
    subplots = plot_data["subplots"]

    # Collect numeric x to define common grid
    all_x = []
    for rep_data in subplots.values():
        x_vals = safe_float_array(rep_data["xdata"])
        if x_vals.size > 0:
            all_x.extend(x_vals.tolist())

    if len(all_x) == 0:
        print(f"⚠️ Skipping {plot_name} (no numeric xdata found)")
        continue

    x_min, x_max = float(np.min(all_x)), float(np.max(all_x))
    if not np.isfinite(x_min) or not np.isfinite(x_max) or x_min == x_max:
        print(f"⚠️ Skipping {plot_name} (invalid x-range)")
        continue

    x_common = np.linspace(x_min, x_max, 200)

    # Interpolate replicas
    replicas = interp_on_common_grid(subplots, x_common)
    if len(replicas) == 0:
        print(f"⚠️ Skipping {plot_name} (no valid replicas)")
        continue

    y_replicas = np.vstack(replicas)  # (n_reps, n_points)

    # Stats
    mean = y_replicas.mean(axis=0)
    sd = y_replicas.std(axis=0, ddof=1) if y_replicas.shape[0] > 1 else np.zeros_like(mean)
    sem = sd / np.sqrt(y_replicas.shape[0]) if y_replicas.shape[0] > 1 else np.zeros_like(mean)

    # --- Plot ---
    plt.figure(figsize=(8, 6))

    # replicas (faint)
    for y in y_replicas:
        plt.plot(x_common, y, color="gray", alpha=0.3, linewidth=1)

    # mean curve (always)
    plt.plot(x_common, mean, color="blue", label="Mean")

    # --- Only for dgde & dgde_unorm/unnorm: SEM + ΔG values ---
    if plot_name in targets:
        # SEM shading
        if np.any(sem):  # only draw if not all zeros
            plt.fill_between(x_common, mean - sem, mean + sem,
                             color="blue", alpha=0.2, label="± SEM")

        # ΔGr (reaction free energy = last min - first min)
        minima_idx = argrelextrema(mean, np.less)[0]
        dGr_mean = dGr_sem = np.nan
        if len(minima_idx) >= 2:
            first_min = int(minima_idx[0])
            last_min = int(minima_idx[-1])
            dGr_mean = float(mean[last_min] - mean[first_min])
            dGr_sem = float(np.sqrt(sem[last_min]**2 + sem[first_min]**2)) if sem.size else 0.0
        
        # ΔG‡ (first local maximum of mean)
        maxima_idx = argrelextrema(mean, np.greater)[0]
        if len(maxima_idx) > 0:
            max_idx = int(maxima_idx[0])   # first local maximum
        else:
            max_idx = int(np.argmax(mean))  # fallback: global maximum
    
        deltaG_mean = float(mean[max_idx]- mean[first_min])
        deltaG_sem = float(np.sqrt(sem[max_idx]**2 + sem[first_min]**2)) if sem.size else 0.0

        # Labels: black, larger, top-right, inside axes, with light bbox
        ax = plt.gca()
        text_kwargs = dict(color="black", fontsize=13, ha="left", va="top",
                           transform=ax.transAxes,
                           bbox=dict(facecolor="white", alpha=0.7, edgecolor="none", boxstyle="round,pad=0.25"))

        ax.text(0.65, 0.88, f"ΔG‡ = {deltaG_mean:.2f} ± {deltaG_sem:.2f}", **text_kwargs)
        if np.isfinite(dGr_mean):
            ax.text(0.65, 0.78, f"ΔGr = {dGr_mean:.2f} ± {dGr_sem:.2f}", **text_kwargs)

        # Print summary
        print(f"Plot: {plot_name}")
        print(f"  ΔG‡ = {deltaG_mean:.2f} ± {deltaG_sem:.2f}")
        if np.isfinite(dGr_mean):
            print(f"  ΔGr = {dGr_mean:.2f} ± {dGr_sem:.2f}")
        print(f"  Saved figure: {plot_name}.png\n")

    # cosmetics
    plt.title(plot_data.get("title", plot_name), fontsize=14)
    plt.xlabel(plot_data.get("xlabel", "x"), fontsize=12)
    plt.ylabel(plot_data.get("ylabel", "y"), fontsize=12)
    plt.legend(fontsize=10, loc="upper left")
    plt.grid(True)
    plt.tight_layout()

    # save
    plt.savefig(f"{plot_name}.png", dpi=300)
    plt.close()

Plot: dgde
  ΔG‡ = 38.73 ± 0.18
  ΔGr = 34.99 ± 0.20
  Saved figure: dgde.png

Plot: dgde_unnorm
  ΔG‡ = 38.73 ± 0.18
  ΔGr = 35.00 ± 0.20
  Saved figure: dgde_unnorm.png

⚠️ Skipping lra_de_st1_1.0 (no numeric xdata found)
⚠️ Skipping lra_de_st2_0.0 (no numeric xdata found)
⚠️ Skipping lra_lra_1.00.0 (no numeric xdata found)
⚠️ Skipping lra_reo_1.00.0 (no numeric xdata found)
